## Preprocessing

In [24]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
columns_to_drop = ["EIN", "NAME"]
application_df.drop(columns_to_drop, axis = 1, inplace = True)

In [3]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_value_counts = application_df["APPLICATION_TYPE"].value_counts()
print(app_type_value_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T29",	"T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_modified_df = application_df[~application_df['CLASSIFICATION'].isin(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() == 1].index)]
application_modified_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1237', 'C1235', 'C7210', 'C4100', 'C1720', 'C2400', 'C1600', 'C1257', 'C1260', 'C2710', 'C0', 'C1267', 'C1256', 'C1234', 'C1246', 'C3200', 'C5200', 'C1236', 'C2190', 'C4200', 'C1248', 'C1370', 'C2600', 'C1580', 'C6100', 'C1820', 'C1900', 'C2570', 'C1570', 'C3700', 'C2500', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C4500', 'C1245', 'C2561', 'C2150']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_application_type_dummies = pd.get_dummies(application_df["APPLICATION_TYPE"])
application_df_affiliation_dummies = pd.get_dummies(application_df["AFFILIATION"])
application_df_classification_dummies = pd.get_dummies(application_df["CLASSIFICATION"])
application_df_use_case_dummies = pd.get_dummies(application_df["USE_CASE"])
application_df_organization_dummies = pd.get_dummies(application_df["ORGANIZATION"])
application_df_income_amt_dummies = pd.get_dummies(application_df["INCOME_AMT"])
application_df_special_considerations_dummies = pd.get_dummies(application_df["SPECIAL_CONSIDERATIONS"])
application_df_updated = pd.concat([application_df_application_type_dummies, application_df_affiliation_dummies, application_df_classification_dummies, application_df_use_case_dummies, application_df_organization_dummies, application_df['STATUS'], application_df_income_amt_dummies, application_df_special_considerations_dummies, application_df["ASK_AMT"], application_df['IS_SUCCESSFUL']], axis=1)
application_df_updated.head() 

,Other,T10,T19,T3,T4,T5,T6,T7,T8,CompanySponsored,...,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M,N,Y,ASK_AMT,IS_SUCCESSFUL
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,5000,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,108590,1
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,5000,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,6692,1
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,142590,1


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_updated.drop(columns=["IS_SUCCESSFUL"])
y = application_df_updated["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=80))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5325 - accuracy: 0.7412
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5327 - accuracy: 0.7424
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5322 - accuracy: 0.7418
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5325 - accuracy: 0.7422
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7424
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5323 - accuracy: 0.7417
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5321 - accuracy: 0.7419
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5324 - accuracy: 0.7422
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7418
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5327 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5784 - accuracy: 0.7237 - 314ms/epoch - 1ms/step
Loss: 0.5783935189247131, Accuracy: 0.7237317562103271


In [28]:
# Export our model to HDF5 file
nn.save('application_model.h5')